# Data Cleaning - Holidays Dataset
## Holidays in France

The Holidays dataset contains information about French public holidays that are defined by the country's labor code. The data is collected from the government's Open Data platform. The dataset includes holidays that occurred up to 20 years in the past and 5 years in the future, relative to the current year (2023). This data can be used for various analyses related to holiday trends, workforce scheduling, and tourism.
We are going to use it in our project in order to check if there a correlation between public holidays and the energy consumption in France.

## Libraries

In [100]:
# importing libraries for data cleaning and analysis :
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Exploring the data

In [144]:
# importing the dataset :

df = pd.read_csv('jours_feries_metropole.csv')
df.head()

,date,annee,zone,nom_jour_ferie
0,2003-01-01,2003,Métropole,1er janvier
1,2003-04-21,2003,Métropole,Lundi de Pâques
2,2003-05-01,2003,Métropole,1er mai
3,2003-05-08,2003,Métropole,8 mai
4,2003-05-29,2003,Métropole,Ascension


In [145]:
# checking the shape of the dataset :
df.shape

(286, 4)

In [146]:
# checking the columns of the dataset :
df.columns

Index(['date', 'annee', 'zone', 'nom_jour_ferie'], dtype='object')

In [147]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286 entries, 0 to 285
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   date            286 non-null    object
 1   annee           286 non-null    int64 
 2   zone            286 non-null    object
 3   nom_jour_ferie  286 non-null    object
dtypes: int64(1), object(3)
memory usage: 9.1+ KB


## Let's begin by Fixing the datetime because it's the most important feature

'date' and 'annee' (year) columns types need to be converted to datetime format. Also we need to fix it as index.

In [148]:
# Convert the "date" column to a datetime format
df['date'] = pd.to_datetime(df['date'])

# Set the "date" column as the index of the DataFrame
#df.set_index('date', inplace=True) Let's not do this for now beacause it will be difficult to merge the datasets

In [149]:
# Drop the "date" and "annee" columns
df.drop(columns=['annee'], inplace=True)

# Check the first few rows of the DataFrame
print(df.head())

        date       zone   nom_jour_ferie
0 2003-01-01  Métropole      1er janvier
1 2003-04-21  Métropole  Lundi de Pâques
2 2003-05-01  Métropole          1er mai
3 2003-05-08  Métropole            8 mai
4 2003-05-29  Métropole        Ascension


In [150]:
df.index

RangeIndex(start=0, stop=286, step=1)

In [151]:
# Creating "Year", "Month", "Day", "Weekday" "Hour" columns : 
df['Year'] = df['date'].dt.year
df['Month'] = df['date'].dt.month
df['Day'] = df['date'].dt.day
df['Weekday'] = df['date'].dt.weekday

In [152]:
# Putting the "Year", "Month", "Day" and "Hour" columns at the beginning of the DataFrame juste after the "date" column :
cols = df.columns.tolist()
cols = cols[:1] + cols[-4:] + cols[1:-4]
df = df[cols]

In [153]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286 entries, 0 to 285
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            286 non-null    datetime64[ns]
 1   Year            286 non-null    int64         
 2   Month           286 non-null    int64         
 3   Day             286 non-null    int64         
 4   Weekday         286 non-null    int64         
 5   zone            286 non-null    object        
 6   nom_jour_ferie  286 non-null    object        
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 15.8+ KB


In [154]:
df.head(5)

,date,Year,Month,Day,Weekday,zone,nom_jour_ferie
0,2003-01-01,2003,1,1,2,Métropole,1er janvier
1,2003-04-21,2003,4,21,0,Métropole,Lundi de Pâques
2,2003-05-01,2003,5,1,3,Métropole,1er mai
3,2003-05-08,2003,5,8,3,Métropole,8 mai
4,2003-05-29,2003,5,29,3,Métropole,Ascension


1) We need explore the data and drop some useless colunms to make the analysis easier.
2) We need to create a flag feature and encode the holidays with the value '1'. 
In our main dataset, we want to add a flag column to check in a specific day is a holiday or not (0 = yes or 1 = 0).

## Dropping Columns (low variance columns)

In [155]:
# checking the values of 'zone' column :
df['zone'].unique()

array(['Métropole'], dtype=object)

In [156]:
# We can drop this column as it has only one value :

df.drop('zone', axis=1, inplace=True)

In [157]:
# Creating a column "Flag Holiday" to indicate if the day is a holiday or not. In this case all the values are 1 :

df['Flag Holiday'] = 1 

In [158]:
# Now we can drop the "nom_jour_ferie" column :
df.drop('nom_jour_ferie', axis=1, inplace=True)

In [159]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286 entries, 0 to 285
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          286 non-null    datetime64[ns]
 1   Year          286 non-null    int64         
 2   Month         286 non-null    int64         
 3   Day           286 non-null    int64         
 4   Weekday       286 non-null    int64         
 5   Flag Holiday  286 non-null    int64         
dtypes: datetime64[ns](1), int64(5)
memory usage: 13.5 KB


In [160]:
df.head(5)

,date,Year,Month,Day,Weekday,Flag Holiday
0,2003-01-01,2003,1,1,2,1
1,2003-04-21,2003,4,21,0,1
2,2003-05-01,2003,5,1,3,1
3,2003-05-08,2003,5,8,3,1
4,2003-05-29,2003,5,29,3,1


## Dropping useless rows

In [161]:
# Printing Datetime min and max values :
print(df['date'].min())
print(df['date'].max()) 
#in the main dataset (energy consumption) we have this time range : 2012-01-01 00:00:00 to 2022-05-31 23:30:00

2003-01-01 00:00:00
2028-12-25 00:00:00


In [162]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286 entries, 0 to 285
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          286 non-null    datetime64[ns]
 1   Year          286 non-null    int64         
 2   Month         286 non-null    int64         
 3   Day           286 non-null    int64         
 4   Weekday       286 non-null    int64         
 5   Flag Holiday  286 non-null    int64         
dtypes: datetime64[ns](1), int64(5)
memory usage: 13.5 KB


So we can drop the useless rows to match the time range of the main dataset

In [163]:
# Defining the start and end dates for your project '2012-01-01' to '2022-05-31' :
# Filter the DataFrame to keep only rows within the specified time range
df = df[(df['date'] >= '2012-01-01') & (df['date'] <= '2022-05-31')]

# Reset the index of the filtered DataFrame
df.reset_index(drop=True, inplace=True)

# Print the filtered DataFrame
print(df)


          date  Year  Month  Day  Weekday  Flag Holiday
0   2012-01-01  2012      1    1        6             1
1   2012-04-09  2012      4    9        0             1
2   2012-05-01  2012      5    1        1             1
3   2012-05-08  2012      5    8        1             1
4   2012-05-17  2012      5   17        3             1
..         ...   ...    ...  ...      ...           ...
110 2022-01-01  2022      1    1        5             1
111 2022-04-18  2022      4   18        0             1
112 2022-05-01  2022      5    1        6             1
113 2022-05-08  2022      5    8        6             1
114 2022-05-26  2022      5   26        3             1

[115 rows x 6 columns]


In [164]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 0 to 114
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          115 non-null    datetime64[ns]
 1   Year          115 non-null    int64         
 2   Month         115 non-null    int64         
 3   Day           115 non-null    int64         
 4   Weekday       115 non-null    int64         
 5   Flag Holiday  115 non-null    int64         
dtypes: datetime64[ns](1), int64(5)
memory usage: 5.5 KB


Now, we have the good time range in this dataset. We don't have the same number of rows and the same start date and end date, that's normal because we only have holidays in this dataset (only 'Flag Holiday" = 1)

## Missing Values

In [165]:
# Checking the number of null values in each column :
df.isnull().sum()

date            0
Year            0
Month           0
Day             0
Weekday         0
Flag Holiday    0
dtype: int64

## Checking Duplicates

In [166]:
# Checking duplicates in the dataset :

df.duplicated().sum()

0

## Let's add all the other days that are not holidays to cover the time range of our project

In [167]:
# Defining the time range
start_date = pd.to_datetime('2012-01-01')
end_date = pd.to_datetime('2022-05-31')

# Creating a DataFrame with all dates in the time range
all_dates = pd.DataFrame(pd.date_range(start=start_date, end=end_date), columns=['date'])

# Merging the original DataFrame and the new DataFrame with all dates
df = pd.merge(all_dates, df, on='date', how='left')

# Replacing missing values in the 'Flag Holiday' column with 0
df['Flag Holiday'].fillna(0, inplace=True)

# Sorting the DataFrame by date
df.sort_values('date', inplace=True)

# Reseting the index of the sorted DataFrame
df.reset_index(drop=True, inplace=True)

# Printing the updated DataFrame
print(df)

           date    Year  Month  Day  Weekday  Flag Holiday
0    2012-01-01  2012.0    1.0  1.0      6.0           1.0
1    2012-01-02     NaN    NaN  NaN      NaN           0.0
2    2012-01-03     NaN    NaN  NaN      NaN           0.0
3    2012-01-04     NaN    NaN  NaN      NaN           0.0
4    2012-01-05     NaN    NaN  NaN      NaN           0.0
...         ...     ...    ...  ...      ...           ...
3799 2022-05-27     NaN    NaN  NaN      NaN           0.0
3800 2022-05-28     NaN    NaN  NaN      NaN           0.0
3801 2022-05-29     NaN    NaN  NaN      NaN           0.0
3802 2022-05-30     NaN    NaN  NaN      NaN           0.0
3803 2022-05-31     NaN    NaN  NaN      NaN           0.0

[3804 rows x 6 columns]


In [168]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3804 entries, 0 to 3803
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          3804 non-null   datetime64[ns]
 1   Year          115 non-null    float64       
 2   Month         115 non-null    float64       
 3   Day           115 non-null    float64       
 4   Weekday       115 non-null    float64       
 5   Flag Holiday  3804 non-null   float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 178.4 KB


In [169]:
# Value counts of the "Flag Holiday" column :

df['Flag Holiday'].value_counts()

0.0    3689
1.0     115
Name: Flag Holiday, dtype: int64

We have successfully added new rows of date that are not holidays.
Let's clean it again!

In [170]:
# Dropping the "Year", "Month", "Day" and "Weekday" columns :

df.drop(['Year', 'Month', 'Day', 'Weekday'], axis=1, inplace=True)

In [171]:
# Convert the "date" column to a datetime format
df['date'] = pd.to_datetime(df['date'])

# Set the "date" column as the index of the DataFrame
#merged_df.set_index('date', inplace=True)

In [172]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3804 entries, 0 to 3803
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          3804 non-null   datetime64[ns]
 1   Flag Holiday  3804 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 59.6 KB


In [173]:
# Creating "Year", "Month", "Day", "Weekday" columns : 
df['Year'] = df['date'].dt.year
df['Month'] = df['date'].dt.month
df['Day'] = df['date'].dt.day
df['Weekday'] = df['date'].dt.weekday

In [175]:
# Putting the "Year", "Month", "Day" columns at the beginning of the DataFrame just after the "date" column :

cols = df.columns.tolist()
cols = cols[:1] + cols[-4:] + cols[1:-4]
df = df[cols]

df.head(5)


,date,Year,Month,Day,Weekday,Flag Holiday
0,2012-01-01,2012,1,1,6,1.0
1,2012-01-02,2012,1,2,0,0.0
2,2012-01-03,2012,1,3,1,0.0
3,2012-01-04,2012,1,4,2,0.0
4,2012-01-05,2012,1,5,3,0.0


In [176]:
# Convert the "Flag Holiday" column to an integer format

df['Flag Holiday'] = df['Flag Holiday'].astype(int)

In [177]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3804 entries, 0 to 3803
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          3804 non-null   datetime64[ns]
 1   Year          3804 non-null   int64         
 2   Month         3804 non-null   int64         
 3   Day           3804 non-null   int64         
 4   Weekday       3804 non-null   int64         
 5   Flag Holiday  3804 non-null   int64         
dtypes: datetime64[ns](1), int64(5)
memory usage: 178.4 KB


In [178]:
df.head(5)

,date,Year,Month,Day,Weekday,Flag Holiday
0,2012-01-01,2012,1,1,6,1
1,2012-01-02,2012,1,2,0,0
2,2012-01-03,2012,1,3,1,0
3,2012-01-04,2012,1,4,2,0
4,2012-01-05,2012,1,5,3,0


In [179]:
# Checking the time period of the dataset :
print(df['date'].min())
print(df['date'].max())

2012-01-01 00:00:00
2022-05-31 00:00:00


Now we have the good time range with all the values that we want !

## Exporting the Dataset to CSV

In [181]:
# Exporting the dataset with all the days to a csv file :
df.to_csv('all_days.csv', index=False)